#### **Análise de negócios**

#### **Observações**

#### **Conteúdo - Bases e Notebook da aula**

Github:  

https://github.com/FIAP/Pos_Tech_DTAT/tree/Analise-de-Negocios/Analise%20de%20Negocios  

Série Histórica de Preços de Combustíveis e de GLP:  

https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-abertos/serie-historica-de-precos-de-combustiveis

#### **Importação de pacotes e bibliotecas**

In [ ]:
# Importar biblioteca completa
import pandas as pd
import os
import requests
from bs4 import BeautifulSoup
import sys
import zipfile  
import io       
import time
import concurrent.futures

# Importar algo especifico de uma biblioteca
from dotenv import load_dotenv
from sqlalchemy import create_engine, text

#### **Funções (def)**

In [ ]:
def encontrar_links(soup):
    
    """
    Encontra dinamicamente os links de download na seção 
    'Combustíveis automotivos'.
    """
    
    # Encontra o cabeçalho <h3> que contém o texto "Combustíveis automotivos"
    heading = soup.find(lambda tag: tag.name == 'h3' and 'Combustíveis automotivos' in tag.get_text())
    
    links_para_baixar = []
    
    if not heading:
        print("Erro: Não foi possível encontrar a seção 'Combustíveis automotivos' no HTML da página")
        return links_para_baixar

    # A lista <ul> com os links é o próximo "irmão" (sibling) da tag <h3>
    ul_tag = heading.find_next_sibling('ul')
    
    if not ul_tag:
        print("Erro: Não foi possível encontrar a lista <ul> após o cabeçalho")
        return links_para_baixar

    # Encontra todas as tags <a> (links) dentro desta lista <ul>
    a_tags = ul_tag.find_all('a')
    
    for a_tag in a_tags:
        url = a_tag.get('href')
        if url:
            links_para_baixar.append(url)
            
    return links_para_baixar

In [ ]:
def processar_arquivo(url, pasta_destino, max_retries, retry_delay):

    """
    Baixa um arquivo com retentativas. Se for .zip, extrai. Se .csv, salva.
    RETORNA uma string de status em vez de imprimir.
    """

    nome_arquivo = os.path.basename(url)
    
    for attempt in range(max_retries):
        try:
            
            if nome_arquivo.endswith('.zip'):
                response = requests.get(url) 
                response.raise_for_status()
                
                with zipfile.ZipFile(io.BytesIO(response.content)) as zf:
                    zf.extractall(pasta_destino)
                    nomes_extraidos = zf.namelist()
                
                return f"[EXTRAÍDO] {nome_arquivo} -> {', '.join(nomes_extraidos)}"

            else:
                caminho_local = os.path.join(pasta_destino, nome_arquivo)
                
                with requests.get(url, stream=True) as r:
                    r.raise_for_status()
                    with open(caminho_local, 'wb') as f:
                        for chunk in r.iter_content(chunk_size=8192): 
                            f.write(chunk)
                            
                return f"[SALVO] {nome_arquivo}"

        except requests.exceptions.RequestException as e:
            if attempt + 1 < max_retries:
                time.sleep(retry_delay)
            else: 
                return f"[FALHA-REDE] {nome_arquivo} após {max_retries} tentativas. Erro: {e}"
        
        except zipfile.BadZipFile:
            return f"[FALHA-ZIP] {nome_arquivo} está corrompido ou não é .zip."

        except Exception as e:
            return f"[FALHA-INESPERADA] {nome_arquivo}. Erro: {e}"

In [ ]:
# Testar a conexão ao banco de dados
def test_connection(engine):

    try:
        with engine.connect() as connection:
            
            # Testar a versão do PostgreSQL
            result = connection.execute(text("SELECT version();"))
            versao = result.fetchone()
            print("✅ Conectado com sucesso:", versao[0])

            # Listar as tabelas no schema público
            result = connection.execute(text("""
                SELECT table_name
                FROM information_schema.tables
                WHERE table_schema = 'anp';
            """))
            tabelas = result.fetchall()
            print("📄 Tabelas no banco:")
            for tabela in tabelas:
                print("-", tabela[0])

    except Exception as e:
        print("❌ Erro ao executar comandos:", e)
        sys.exit()


#### **Credenciais**

In [ ]:
load_dotenv()

# Credenciais do PostgreSQL
usuario_pg = os.getenv("POSTGRES_USER")
senha_pg = os.getenv("POSTGRES_PASSWORD")
host_pg = os.getenv("POSTGRES_HOST")
porta_pg = os.getenv("POSTGRES_PORT")
banco_pg = os.getenv("POSTGRES_DB")

#### **Variaveis**

In [ ]:
# Número máximo de tentativas por arquivo
max_retries = 5

# Segundos de espera entre as tentativas
retry_delay = 5  

# Maximo de threads
max_workers = 20

# URL da página para extrair os links
page_url = 'https://www.gov.br/anp/pt-br/centrais-de-conteudo/dados-abertos/serie-historica-de-precos-de-combustiveis'

# Pasta onde os arquivos serão baixados
download_dir = 'arquivos_combustiveis_automotivos'

# Validar download dos arquivos ANP
baixar_arquivos_anp = 'n'

#### **Aula 1 - Processos e formas de análise**

#### **Aula 2 - Ligação com bancos de dados**

In [ ]:
# Criar engine com banco 
engine = create_engine(f"postgresql+psycopg2://{usuario_pg}:{senha_pg}@{host_pg}:{porta_pg}/{banco_pg}")

# Testar a conexão
test_connection(engine)

In [ ]:
# Baixar os arquivos da ANP

if baixar_arquivos_anp.lower() == 'n':
    print(f'Etapa de carregar os dados do Github para o PostgreSQL não realizada pois a variavel baixar_arquivos_anp é `n`')

else:

    # 1. Criar a pasta de download
    os.makedirs(download_dir, exist_ok=True)

    # 2. Baixar o HTML da página da ANP
    print(f"Acessando a página: {page_url}")

    try:
        response = requests.get(page_url)
        response.raise_for_status()
        conteudo_html = response.text
        
    except requests.exceptions.RequestException as e:
        print(f"Erro fatal ao acessar a página da ANP: {e}")
        print("Verifique sua conexão com a internet ou se a URL da ANP mudou")
        sys.exit(1) 

    # 3. Analisar (parse) o HTML
    soup = BeautifulSoup(conteudo_html, 'html.parser')

    # 4. Encontrar os links
    links = encontrar_links(soup)

    if not links:
        print("Nenhum link encontrado para baixar")

    else:
        print(f"Encontrados {len(links)} arquivos para baixar na seção 'Combustíveis automotivos'")
        print()

        # 5. Processar (baixar ou extrair) cada arquivo EM PARALELO
        with concurrent.futures.ThreadPoolExecutor(max_workers=max_workers) as executor:
            print(f"Iniciando downloads com até {max_workers} threads em paralelo...")
            print("-" * 70)
            future_to_url = {}

            for url in links:
                future = executor.submit(processar_arquivo, 
                                        url, 
                                        download_dir, 
                                        max_retries, 
                                        retry_delay)
                
                future_to_url[future] = url

            num_concluidos = 0

            for future in concurrent.futures.as_completed(future_to_url):
                num_concluidos += 1
                url = future_to_url[future]
                
                try:
                    status_message = future.result()
                    print(f"({num_concluidos}/{len(links)}) {status_message}")
                    
                except Exception as exc:
                    print(f"({num_concluidos}/{len(links)}) [FALHA-GERAL] {url} gerou uma exceção: {exc}")
                    
        print("-" * 70)
        print()
        print("Download e processamento de todos os arquivos concluído")
        print(f"Os arquivos estão salvos em: {os.path.abspath(download_dir)}")

In [21]:
# Importações necessárias para esta célula
import os
import pandas as pd
import psycopg2 # Driver PostgreSQL
import io       # Para buffer em memória
import csv      # Para opções de formatação CSV

# --- Configurações ---
nome_tabela = 'anp.preco_combustivel' 
chunksize = 100000                

colunas_tabela_pg = [
    'regiao', 'estado', 'municipio', 'revenda', 'cnpj', 'nome_rua', 
    'numero_rua', 'complemento', 'bairro', 'cep', 'produto', 
    'data_coleta', 'valor_venda', 'unidade_medida', 'bandeira'
]
colunas_sql_copy = ', '.join(f'"{col}"' for col in colunas_tabela_pg) 

mapeamento_colunas_csv_para_pg = {
    'Regiao - Sigla': 'regiao', 'Estado - Sigla': 'estado', 'Municipio': 'municipio',
    'Revenda': 'revenda', 'CNPJ da Revenda': 'cnpj', 'Nome da Rua': 'nome_rua',
    'Numero Rua': 'numero_rua', 'Complemento': 'complemento', 'Bairro': 'bairro',
    'Cep': 'cep', 'Produto': 'produto', 'Data da Coleta': 'data_coleta',
    'Valor de Venda': 'valor_venda', 'Unidade de Medida': 'unidade_medida',
    'Bandeira': 'bandeira'
}

download_dir = 'arquivos_combustiveis_automotivos' 
carregar_tabela = 's' 

# --- Conexão (psycopg2) ---
conn_str_psycopg = f"dbname='{banco_pg}' user='{usuario_pg}' password='{senha_pg}' host='{host_pg}' port='{porta_pg}'"

# --- Lógica Principal de Carga ---

if carregar_tabela.lower() == 'n':
    print(f'Etapa de carregar os dados para o PostgreSQL não realizada pois a variavel carregar_tabela é `n`')
else:
    print(f"\nIniciando carga de dados via COPY para a tabela '{nome_tabela}' a partir de '{download_dir}'")
    
    conn_psycopg = None 
    cursor = None       
    
    try:
        print("Conectando ao PostgreSQL via psycopg2...")
        conn_psycopg = psycopg2.connect(conn_str_psycopg)
        conn_psycopg.autocommit = False 
        cursor = conn_psycopg.cursor()
        print("✅ Conectado com sucesso.")

        print(f"\nProcurando arquivos .csv em '{download_dir}'...")
        arquivos_csv = sorted([f for f in os.listdir(download_dir) if f.endswith('.csv')]) 
        
        if not arquivos_csv:
             print("Nenhum arquivo .csv encontrado na pasta. Carga não realizada.")
        else:
            print(f"Encontrados {len(arquivos_csv)} arquivos .csv. Iniciando carga via COPY...")
            arquivos_processados = 0
            arquivos_com_erro = 0

            for nome_arquivo_csv in arquivos_csv:
                caminho_completo = os.path.join(download_dir, nome_arquivo_csv)
                print(f"\n--- Processando arquivo: {nome_arquivo_csv} ({arquivos_processados + arquivos_com_erro + 1}/{len(arquivos_csv)}) ---")
                
                # --->>> LÓGICA DE TENTATIVA DE ENCODING <<<---
                chunk_iterator = None
                encodings_to_try = ['utf-8', 'latin-1'] # Lista de codificações a tentar
                
                for encoding_attempt in encodings_to_try:
                    try:
                        print(f"  Tentando ler com encoding: {encoding_attempt}...")
                        chunk_iterator = pd.read_csv(
                            caminho_completo, 
                            chunksize=chunksize, 
                            low_memory=False, 
                            sep=';',              
                            encoding=encoding_attempt, # Tenta a codificação atual
                            decimal=',',          
                            parse_dates=['Data da Coleta'], 
                            dayfirst=True         
                        )
                        print(f"  Sucesso ao ler com {encoding_attempt}.")
                        break # Sai do loop de encodings se a leitura funcionar
                    
                    except UnicodeDecodeError:
                        print(f"  Falha ao ler com {encoding_attempt}. Tentando próximo...")
                        if encoding_attempt == encodings_to_try[-1]: # Se foi a última tentativa
                            raise # Re-levanta o erro se nenhuma codificação funcionou
                    
                    except FileNotFoundError: # Captura logo se o arquivo não existe
                         raise # Re-levanta para ser tratado no bloco externo
                    
                    except Exception as e_read: # Outros erros de leitura
                        print(f"  Erro inesperado ao tentar ler com {encoding_attempt}: {e_read}")
                        raise # Re-levanta outros erros

                # Se chunk_iterator ainda for None, algo deu muito errado antes mesmo de tentar ler
                if chunk_iterator is None:
                     raise Exception("Não foi possível iniciar a leitura do arquivo CSV.")
                # --->>> FIM DA LÓGICA DE ENCODING <<<---

                # Processamento dos chunks (continua como antes)
                try:
                    total_chunks = 0
                    for i, chunk in enumerate(chunk_iterator):
                        total_chunks = i + 1
                        # (O print do chunk foi removido daqui para evitar poluir o log se houver erro depois)
                        
                        # 1. Renomear colunas
                        try:
                            chunk_renamed = chunk.rename(columns=mapeamento_colunas_csv_para_pg)
                        except Exception as e_rename:
                            print(f"    ERRO ao renomear colunas no chunk {total_chunks}: {e_rename}")
                            raise 

                        # 2. Reindexar
                        chunk_reordered = chunk_renamed.reindex(columns=colunas_tabela_pg)

                        # 3. Preparar buffer
                        buffer = io.StringIO()
                        chunk_reordered.to_csv(buffer, index=False, header=False, sep=',', 
                                               na_rep='\\N', quoting=csv.QUOTE_MINIMAL, 
                                               date_format='%Y-%m-%d')
                        buffer.seek(0) 

                        # 4. Comando COPY
                        print(f"  Carregando chunk {total_chunks}...") # Printa só antes do COPY
                        sql_copy_command = f"""COPY {nome_tabela} ({colunas_sql_copy}) FROM STDIN WITH (FORMAT CSV, HEADER FALSE, NULL '\\N', DELIMITER ',')"""
                        cursor.copy_expert(sql_copy_command, buffer)
                        
                    print(f"  -> Arquivo {nome_arquivo_csv} carregado ({total_chunks} chunks).")
                    conn_psycopg.commit() 
                    arquivos_processados += 1

                # Tratamento de erros específicos do arquivo/chunk
                except pd.errors.EmptyDataError:
                    print(f"  AVISO: Arquivo {nome_arquivo_csv} está vazio ou tornou-se vazio após leitura. Pulando.")
                    conn_psycopg.rollback() 
                    arquivos_com_erro += 1 
                except FileNotFoundError: # Pouco provável aqui, mas seguro ter
                    print(f"❌ Erro: Arquivo {nome_arquivo_csv} não encontrado durante processamento dos chunks.")
                    conn_psycopg.rollback()
                    arquivos_com_erro += 1
                except Exception as e_file:
                    # Captura erros ocorridos DURANTE o processamento dos chunks (rename, reindex, to_csv, copy_expert)
                    print(f"❌ Erro ao processar o arquivo {nome_arquivo_csv} (no chunk {total_chunks}): {e_file}")
                    print(f"    Verifique o mapeamento, tipos de dados (especialmente datas e decimais).")
                    conn_psycopg.rollback() 
                    arquivos_com_erro += 1
            
            # ... (Restante do código: Resumo final, Fechar conexão, Adicionar metadados) ...
            print("-" * 70)
            print(f"\n✅ Carga via COPY concluída.")
            print(f"  Arquivos processados com sucesso: {arquivos_processados}")
            print(f"  Arquivos com erro/vazios: {arquivos_com_erro}")

    except psycopg2.Error as db_err:
        print(f"❌ Erro de conexão ou execução no PostgreSQL (psycopg2): {db_err}")
        if conn_psycopg:
            conn_psycopg.rollback() 
    except Exception as e:
        # Captura erros ocorridos ANTES do loop de chunks (ex: falha ao ler o 1o chunk com ambos encodings)
        print(f"❌ Erro inesperado durante a carga dos dados (possivelmente ao iniciar leitura de {nome_arquivo_csv}): {e}")
        if conn_psycopg:
            conn_psycopg.rollback()
    finally:
        if cursor:
            cursor.close()
        if conn_psycopg:
            conn_psycopg.close()
            print("\nConexão psycopg2 fechada.")
            
    print("\nProcesso de carga finalizado.")


Iniciando carga de dados via COPY para a tabela 'anp.preco_combustivel' a partir de 'arquivos_combustiveis_automotivos'
Conectando ao PostgreSQL via psycopg2...
✅ Conectado com sucesso.

Procurando arquivos .csv em 'arquivos_combustiveis_automotivos'...
Encontrados 43 arquivos .csv. Iniciando carga via COPY...

--- Processando arquivo: Preços semestrais - AUTOMOTIVOS_2023.01.csv (1/43) ---
  Tentando ler com encoding: utf-8...
  Sucesso ao ler com utf-8.
  Carregando chunk 1...
  Carregando chunk 2...
  Carregando chunk 3...
  Carregando chunk 4...
  Carregando chunk 5...
  -> Arquivo Preços semestrais - AUTOMOTIVOS_2023.01.csv carregado (5 chunks).

--- Processando arquivo: Preços semestrais - AUTOMOTIVOS_2023.02.csv (2/43) ---
  Tentando ler com encoding: utf-8...
  Sucesso ao ler com utf-8.
  Carregando chunk 1...
  Carregando chunk 2...
  Carregando chunk 3...
  Carregando chunk 4...
  Carregando chunk 5...
  -> Arquivo Preços semestrais - AUTOMOTIVOS_2023.02.csv carregado (5 chun